#### 주의!!

이 실습은 가급적 NVIDIA GPU가 설치된 컴퓨터 환경이거나 Google Colab에서 진행해주세요.

<a target="_blank" href="https://colab.research.google.com/github/Taekyoon/keras_tutorial/blob/main/230321/230321_gpt3_fewshot_nsmc_alpaca.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## 환경 준비 
(Google Colab 환경에서 사용하세요)

In [1]:
!pip install transformers
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 33.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.8/212.8 KB 10.3 MB/s eta 0:00:00


## 데이터 다운로드
(Google Colab 환경에서 사용하세요)

In [2]:
!mkdir -p data_in/KOR/naver_movie
!wget https://raw.githubusercontent.com/NLP-kr/tensorflow-ml-nlp-tf2/master/7.PRETRAIN_METHOD/data_in/KOR/naver_movie/ratings_train.txt \
              -O data_in/KOR/naver_movie/ratings_train.txt
!wget https://raw.githubusercontent.com/NLP-kr/tensorflow-ml-nlp-tf2/master/7.PRETRAIN_METHOD/data_in/KOR/naver_movie/ratings_test.txt \
              -O data_in/KOR/naver_movie/ratings_test.txt

--2023-03-21 11:01:25--  https://raw.githubusercontent.com/NLP-kr/tensorflow-ml-nlp-tf2/master/7.PRETRAIN_METHOD/data_in/KOR/naver_movie/ratings_train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14628807 (14M) [text/plain]
Saving to: ‘data_in/KOR/naver_movie/ratings_train.txt’

data_in/KOR/naver_m 100%[===================>]  13.95M  --.-KB/s    in 0.07s   

2023-03-21 11:01:25 (198 MB/s) - ‘data_in/KOR/naver_movie/ratings_train.txt’ saved [14628807/14628807]

--2023-03-21 11:01:25--  https://raw.githubusercontent.com/NLP-kr/tensorflow-ml-nlp-tf2/master/7.PRETRAIN_METHOD/data_in/KOR/naver_movie/ratings_test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.gi

In [14]:
import os
from transformers import AutoModelForCausalLM, AutoTokenizer

import torch
import pandas as pd
import numpy as np

import re

import random
from random import sample

from tqdm import tqdm

In [4]:
# os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [5]:
SEED_NUM = 1234
random.seed(SEED_NUM)

## 퓨샷 러닝을 위한 네이버 영화 리뷰 모델 구성


In [6]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/polyglot-ko-5.8b")

In [7]:
cls_model = AutoModelForCausalLM.from_pretrained("beomi/KoAlpaca-Polyglot",
                                                 torch_dtype=torch.float16,
                                                 device_map='sequential',
                                                 low_cpu_mem_usage=True).cuda()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
cls_model.config.max_length = 2048
cls_model.config.pad_token_id = 0

## 퓨샷 러닝을 위한 네이버 영화 리뷰 데이터 구성

In [9]:
# 데이터 전처리 준비
DATA_IN_PATH = './data_in/KOR'
DATA_OUT_PATH = './data_out/KOR'

DATA_TRAIN_PATH = os.path.join(DATA_IN_PATH, 'naver_movie', 'ratings_train.txt')
DATA_TEST_PATH = os.path.join(DATA_IN_PATH, 'naver_movie', 'ratings_test.txt')

train_data = pd.read_csv(DATA_TRAIN_PATH, header = 0, delimiter = '\t', quoting = 3)
train_data = train_data.dropna()

In [10]:
print('데이터 positive 라벨: ', '긍정')
print('데이터 negative 라벨: ', '부정')

데이터 positive 라벨:  긍정
데이터 negative 라벨:  부정


In [11]:
print('학습 예시 케이스 구조: ', '문장: 오늘 기분이 좋아\n감정: 긍정\n')

학습 예시 케이스 구조:  문장: 오늘 기분이 좋아
감정: 긍정



In [12]:
print('gpt3 최대 토큰 길이: ', cls_model.config.max_position_embeddings)

gpt3 최대 토큰 길이:  2048


In [15]:
sent_lens = [len(tokenizer(s).input_ids) for s in tqdm(train_data['document'])]

print('Few shot 케이스 토큰 평균 길이: ', np.mean(sent_lens))
print('Few shot 케이스 토큰 최대 길이: ', np.max(sent_lens))
print('Few shot 케이스 토큰 길이 표준편차: ',np.std(sent_lens))
print('Few shot 케이스 토큰 길이 80 퍼센타일: ',np.percentile(sent_lens, 80))

100%|██████████| 149995/149995 [00:27<00:00, 5476.22it/s]

Few shot 케이스 토큰 평균 길이:  20.22912763758792
Few shot 케이스 토큰 최대 길이:  280
Few shot 케이스 토큰 길이 표준편차:  16.48828728915166
Few shot 케이스 토큰 길이 80 퍼센타일:  27.0


In [16]:
train_fewshot_data = []

for train_sent, train_label in tqdm(train_data[['document', 'label']].values):
    tokens = tokenizer(train_sent).input_ids

    if len(tokens) <= 25:
        train_fewshot_data.append((train_sent, train_label))

100%|██████████| 149995/149995 [00:19<00:00, 7612.58it/s]


## 네이버 영화 리뷰 데이터를 활용한 퓨샷 러닝 및 평가

In [25]:
test_data = pd.read_csv(DATA_TEST_PATH, header=0, delimiter='\t', quoting=3)
test_data = test_data.dropna()
test_data.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [26]:
# 평가 데이터 수
sample_size = 500

# 평가에서 활용할 few-shot 예제를 묶음으로 저장
train_fewshot_samples = []

for _ in range(sample_size):
    # few-shot 예제를 10개씩 묶음
    fewshot_examples = sample(train_fewshot_data, 10)
    train_fewshot_samples.append(fewshot_examples)

if sample_size < len(test_data['id']):
    test_data = test_data.sample(sample_size, random_state=SEED_NUM)

In [32]:
def build_prompt_text(sent):
    return "문장: " + sent + '\n감정:'

def clean_text(sent):
    sent_clean = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", "", sent)
    return sent_clean

def generate_fewshot_example(data, with_label=True):
    example_text, example_label = data
    # 텍스트 전처리
    cleaned_example_text = clean_text(example_text)
    # Prompt 형식 구성
    fewshot_example_text = build_prompt_text(cleaned_example_text)
    # Label 추가
    if with_label:
      fewshot_example_text += ' 긍정' if example_label == 1 else ' 부정' + '\n'
    
    return fewshot_example_text

def predict_by_generation(prompt_text):
    # 토큰화 및 인덱싱
    tokens = tokenizer(prompt_text, return_tensors="pt")
    token_ids, attn_mask = tokens.input_ids.cuda(), tokens.attention_mask.cuda()
    # 텍스트 생성
    gen_tokens = cls_model.generate(input_ids=token_ids, attention_mask=attn_mask,
                                    max_new_tokens=1, pad_token_id=0)
    # 인덱스 복호화
    pred = tokenizer.batch_decode(gen_tokens[:, -1])[0].strip()
    
    return pred

In [33]:
real_labels = []
pred_tokens = []

total_len = len(test_data[['document','label']].values)

for i, row in tqdm(enumerate(test_data[['document','label']].values), total=total_len):
    prompt_text = ''

    for ex in train_fewshot_samples[i]:
        prompt_text += generate_fewshot_example(ex)

    prompt_text += generate_fewshot_example(row, with_label=False)

    pred = predict_by_generation(prompt_text)

    pred_tokens.append(pred)
    real_labels.append('긍정' if row[1] == 1 else '부정')

100%|██████████| 500/500 [01:31<00:00,  5.47it/s]


In [35]:
accuracy_match = [p == t for p, t in zip(pred_tokens, real_labels)]
accuracy = len([m for m in accuracy_match if m]) / len(real_labels)

print(accuracy)

0.762


In [36]:
def build_prompt_text(sent):
    return '다음 문장은 긍정일까요 부정일까요?\n' + sent + '\n정답:'

real_labels = []
pred_tokens = []

total_len = len(test_data[['document','label']].values)

for i, row in tqdm(enumerate(test_data[['document','label']].values), total=total_len):
    prompt_text = ''

    for ex in train_fewshot_samples[i]:
        prompt_text += generate_fewshot_example(ex)

    prompt_text += generate_fewshot_example(row, with_label=False)

    pred = predict_by_generation(prompt_text)

    pred_tokens.append(pred)
    real_labels.append('긍정' if row[1] == 1 else '부정')

100%|██████████| 500/500 [02:05<00:00,  3.99it/s]


In [37]:
accuracy_match = [p == t for p, t in zip(pred_tokens, real_labels)]
accuracy = len([m for m in accuracy_match if m]) / len(real_labels)

print(accuracy)

0.742
